# Performance loops with `numba`

This first bit is the same as the *Intro_to_scientific_computing* notebook.

In [1]:
layers = [0.23, 0.34, 0.45, 0.25, 0.23, 0.35]

In [2]:
uppers = layers[:-1]
lowers = layers[1:]

In [3]:
rcs = []
for pair in zip(lowers, uppers):
    rc = (pair[1] - pair[0]) / (pair[1] + pair[0])
    rcs.append(rc)

In [4]:
rcs

[-0.1929824561403509,
 -0.1392405063291139,
 0.28571428571428575,
 0.04166666666666665,
 -0.2068965517241379]

## Functions

Definition, inputs, side-effects, returning, scope, docstrings

In [5]:
# Exercise
def compute_rc(layers):
    """
    Computes reflection coefficients given
    a list of layer impedances.
    """
    uppers = layers[:-1]
    lowers = layers[1:]
    rcs = []
    for pair in zip(lowers, uppers):
        rc = (pair[1] - pair[0]) / (pair[1] + pair[0])
        rcs.append(rc)
    return rcs

In [6]:
compute_rc(layers)

[-0.1929824561403509,
 -0.1392405063291139,
 0.28571428571428575,
 0.04166666666666665,
 -0.2068965517241379]

Put in a file and import into a new notebook

## Numpy

**Before continuing, do some basic NumPy array stuff in the interpreter.**

Let's make a really big 'log' from random numbers:

In [7]:
import numpy as np  # Just like importing file

In [8]:
biglog = np.random.random(10000000)
%timeit compute_rc(biglog)

1 loops, best of 3: 5.65 s per loop


Note that the log has to be fairly big for the benchmarking to work properly, because otherwise the CPU caches the computation and this skews the results.

Now we can re-write our function using arrays instead of lists.

In [9]:
# Exercise
def compute_rc_vector(layers):
    uppers = layers[:-1]
    lowers = layers[1:]
    return (lowers - uppers) / (uppers + lowers)

In [10]:
%timeit compute_rc_vector(biglog)

10 loops, best of 3: 90 ms per loop


60 times faster on my machine!

## Aside: more performance with `numba`

In [11]:
from numba import jit

In [12]:
@jit
def compute_rc_numba(layers):
    uppers = layers[:-1]
    lowers = layers[1:]
    return (lowers - uppers) / (uppers + lowers)

In [13]:
%timeit compute_rc_numba(biglog)

The slowest run took 12.52 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 88.2 ms per loop


OK, we'll make a fake example.

In [14]:
def compute_rc_slow(layers):
    uppers = layers[:-1]
    lowers = layers[1:]
    rcs = np.zeros_like(uppers)
    for i in range(rcs.size):
        rcs[i] = (lowers[i] - uppers[i]) / (uppers[i] + lowers[i])
    return rcs

In [15]:
%timeit compute_rc_slow(biglog)

1 loops, best of 3: 6.54 s per loop


In [16]:
@jit
def compute_rc_faster(layers):
    uppers = layers[:-1]
    lowers = layers[1:]
    rcs = np.zeros_like(uppers)
    for i in range(rcs.size):
        rcs[i] = (lowers[i] - uppers[i]) / (uppers[i] + lowers[i])
    return rcs

In [17]:
%timeit compute_rc_faster(biglog)

The slowest run took 29.06 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 64.4 ms per loop


However, you can't speed up our original list-based function this way.

In [18]:
@jit
def compute_rc_hopeful(layers):
    """
    Computes reflection coefficients given
    a list of layer impedances.
    """
    uppers = layers[:-1]
    lowers = layers[1:]
    rcs = []
    for pair in zip(lowers, uppers):
        rc = (pair[1] - pair[0]) / (pair[1] + pair[0])
        rcs.append(rc)
    return rcs

In [19]:
%timeit compute_rc_hopeful(biglog)

1 loops, best of 3: 5.33 s per loop


<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>